In [1]:
# dados fin
import yfinance as yf

start = '2021-01-01'
end = '2023-06-06'

data = yf.download('AAPL', 
                   start=start,
                   end=end)

data.head()

data.to_csv('data_/AAPL.csv')

[*********************100%***********************]  1 of 1 completed


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd

# Carregando os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,133.520004,133.610001,126.760002,129.410004,127.503662,143301900
1,2021-01-05,128.889999,131.740005,128.429993,131.009995,129.080078,97664900
2,2021-01-06,127.720001,131.050003,126.379997,126.599998,124.735031,155088000
3,2021-01-07,128.360001,131.630005,127.860001,130.919998,128.991425,109578200
4,2021-01-08,132.429993,132.630005,130.229996,132.050003,130.104767,105158200


In [4]:
def calculate_moving_average(data, window):
    return data['Close'].rolling(window=window).mean()

# Vamos testar para diferentes períodos de média móvel (por exemplo, de 5 a 50 dias)
moving_averages = []
for window in range(5, 51):
    moving_averages.append(calculate_moving_average(data, window))


In [5]:
def calculate_distance_from_moving_average(data, moving_average):
    return abs(data['Close'] - moving_average)

distance_from_moving_averages = []
for moving_average in moving_averages:
    distance_from_moving_averages.append(calculate_distance_from_moving_average(data, moving_average))

# Vamos calcular a média das distâncias para cada período
mean_distances = [dist.mean() for dist in distance_from_moving_averages]


In [6]:
best_window = mean_distances.index(min(mean_distances)) + 5  # +5 pois começamos a partir da janela 5

print(f'A melhor média móvel é de {best_window} dias.')

A melhor média móvel é de 5 dias.


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.linear_model import LinearRegression

# Carregando os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Definindo a função para calcular a distância da média móvel simples
def calculate_distance_from_simple_moving_average(data, window):
    moving_average = data['Close'].rolling(window=window).mean()
    return abs(data['Close'] - moving_average)

# Criação de um array de períodos de média móvel a serem testados
window_sizes = np.arange(5, 51)

# Preparação dos dados para validação cruzada
X = data[['Close']]
y = data['Close']

# Definindo o modelo de regressão linear
model = LinearRegression()

# Inicializando as listas para armazenar os resultados
best_window = None
best_mse = float('inf')
mse_values = []

# Realizando a busca pelo melhor período de média móvel
for window in window_sizes:
    mse = -np.mean(calculate_distance_from_simple_moving_average(data, window)**2)
    mse_values.append(mse)
    if mse < best_mse:
        best_mse = mse
        best_window = window

print(f"A melhor média móvel é de {best_window} dias, com MSE de {best_mse:.2f}.")

A melhor média móvel é de 50 dias, com MSE de -105.27.


In [8]:
# Calculando a melhor média móvel com o período encontrado pelo GridSearch
best_moving_average = data['Close'].rolling(window=best_window).mean()

# Criando colunas para sinal de compra (1), sinal de venda (-1) e sinal neutro (0)
data['Buy_Signal'] = np.where(data['Close'] > best_moving_average, 1, 0)
data['Sell_Signal'] = np.where(data['Close'] < best_moving_average, -1, 0)

# Calculando o lucro e a perda (PnL) da estratégia
data['PnL'] = data['Close'].pct_change() * data['Buy_Signal'].shift(-1)

# Calculando o retorno cumulativo
data['Cumulative_Return'] = (1 + data['PnL']).cumprod()

# Visualizando os resultados
print(data[['Close', 'Buy_Signal', 'Sell_Signal', 'PnL', 'Cumulative_Return']].tail(10))


                 Close  Buy_Signal  Sell_Signal  PnL  Cumulative_Return
Date                                                                   
2022-12-16  134.509995           0           -1 -0.0           2.397015
2022-12-19  132.369995           0           -1 -0.0           2.397015
2022-12-20  132.300003           0           -1 -0.0           2.397015
2022-12-21  135.449997           0           -1  0.0           2.397015
2022-12-22  132.229996           0           -1 -0.0           2.397015
2022-12-23  131.860001           0           -1 -0.0           2.397015
2022-12-27  130.029999           0           -1 -0.0           2.397015
2022-12-28  126.040001           0           -1 -0.0           2.397015
2022-12-29  129.610001           0           -1  0.0           2.397015
2022-12-30  129.929993           0           -1  NaN                NaN


In [9]:
# Cálculo do retorno anualizado
days_per_year = 252  # Supondo que há 252 dias úteis em um ano
total_trading_days = data['PnL'].count()
annualized_return = ((data['Cumulative_Return'].iloc[-1]) ** (days_per_year / total_trading_days)) - 1

# Cálculo do risco livre (pode ser ajustado de acordo com a taxa livre de risco atual)
risk_free_rate = 0.03  # Exemplo: 3% ao ano

# Cálculo do índice de Sharpe
daily_returns = data['PnL']
daily_risk_free_rate = (1 + risk_free_rate) ** (1 / days_per_year) - 1
daily_volatility = daily_returns.std()
sharpe_ratio = (daily_returns.mean() - daily_risk_free_rate) / daily_volatility

# Cálculo do drawdown máximo
cumulative_returns = data['Cumulative_Return']
previous_peaks = cumulative_returns.cummax()
drawdowns = cumulative_returns / previous_peaks - 1
max_drawdown = drawdowns.min()

# Visualizando as métricas
print(f"Retorno anualizado: {annualized_return:.2%}")
print(f"Índice de Sharpe: {sharpe_ratio:.2f}")
print(f"Drawdown máximo: {max_drawdown:.2%}")


Retorno anualizado: nan%
Índice de Sharpe: 0.14
Drawdown máximo: -11.03%


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Preparação dos dados para o modelo de machine learning
X = data[['Close']].shift(1).dropna()  # Usamos o preço de fechamento do dia anterior como feature
y = data['Buy_Signal'].shift(-1).dropna()  # O sinal de compra do próximo dia é o rótulo

# Divisão dos dados em conjuntos de treinamento e teste
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Criação e treinamento do modelo de Regressão Logística
model = LogisticRegression()
model.fit(X_train, y_train)

# Realizando as previsões para o conjunto de teste
y_pred = model.predict(X_test)

# Avaliando o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
classification_report_output = classification_report(y_test, y_pred)

# Visualizando os resultados
print(f"Acurácia do modelo: {accuracy:.2f}")
print("Relatório de Classificação:\n", classification_report_output)


Acurácia do modelo: 0.72
Relatório de Classificação:
               precision    recall  f1-score   support

         0.0       0.93      0.62      0.74        65
         1.0       0.57      0.92      0.70        36

    accuracy                           0.72       101
   macro avg       0.75      0.77      0.72       101
weighted avg       0.80      0.72      0.73       101



In [11]:
from sklearn.svm import SVC

# Criação e treinamento do modelo SVM
model_svm = SVC(kernel='linear', C=1.0)
model_svm.fit(X_train, y_train)

# Realizando as previsões para o conjunto de teste
y_pred_svm = model_svm.predict(X_test)

# Avaliando o desempenho do modelo SVM
accuracy_svm = accuracy_score(y_test, y_pred_svm)
classification_report_output_svm = classification_report(y_test, y_pred_svm)

# Visualizando os resultados do modelo SVM
print(f"Acurácia do modelo SVM: {accuracy_svm:.2f}")
print("Relatório de Classificação do modelo SVM:\n", classification_report_output_svm)


Acurácia do modelo SVM: 0.72
Relatório de Classificação do modelo SVM:
               precision    recall  f1-score   support

         0.0       0.91      0.63      0.75        65
         1.0       0.57      0.89      0.70        36

    accuracy                           0.72       101
   macro avg       0.74      0.76      0.72       101
weighted avg       0.79      0.72      0.73       101



In [12]:
#!pip install tensorflow

In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Carregar os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Preparação dos dados
data['Target'] = np.where(data['Close'].shift(-1) > data['Close'], 1, -1)

X = data[['Close']]
y = data['Target']

# Divisão dos dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Criação e treinamento do modelo de Rede Neural
model_nn = Sequential([
    Dense(64, activation='relu', input_shape=(1,)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_nn.compile(optimizer='adam', loss='mean_squared_error')  # Usando MSE para regressão
model_nn.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Realizando as previsões para o conjunto de teste
y_pred_nn = model_nn.predict(X_test)

# Calculando as métricas de regressão
mae_nn = mean_absolute_error(y_test, y_pred_nn)
mse_nn = mean_squared_error(y_test, y_pred_nn)

# Visualizando os resultados do modelo de Rede Neural
print(f"Erro Médio Absoluto do modelo de Rede Neural: {mae_nn:.2f}")
print(f"Erro Médio Quadrático do modelo de Rede Neural: {mse_nn:.2f}")

4/4 [==============================] - 0s 3ms/step
Erro Médio Absoluto do modelo de Rede Neural: 1.00
Erro Médio Quadrático do modelo de Rede Neural: 1.00


In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.arima.model import ARIMA

# Carregar os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Preparação dos dados
data['Target'] = np.where(data['Close'].shift(-1) > data['Close'], 1, -1)

X = data[['Close']]
y = data['Target']

# Divisão dos dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Criação e treinamento do modelo ARIMA
model_arima = ARIMA(X_train, order=(5, 1, 0))  # Ordem (p, d, q) ajustável conforme necessário
model_arima_fit = model_arima.fit()

# Realizando as previsões para o conjunto de teste
y_pred_arima = model_arima_fit.forecast(steps=len(X_test))

# Calculando as métricas de regressão
mae_arima = mean_absolute_error(y_test, y_pred_arima)
mse_arima = mean_squared_error(y_test, y_pred_arima)

# Visualizando os resultados do modelo ARIMA
print(f"Erro Médio Absoluto do modelo ARIMA: {mae_arima:.2f}")
print(f"Erro Médio Quadrático do modelo ARIMA: {mse_arima:.2f}")


Erro Médio Absoluto do modelo ARIMA: 165.42
Erro Médio Quadrático do modelo ARIMA: 27366.36


c:\Users\Caíque Miranda\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Caíque Miranda\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Caíque Miranda\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Caíque Miranda\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: Val

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

# Carregar os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Preparação dos dados
data['Target'] = np.where(data['Close'].shift(-1) > data['Close'], 1, -1)

X = data[['Close']]
y = data['Target']

# Divisão dos dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Preparação dos dados para o modelo LSTM
X_train_lstm = X_train.values.reshape(-1, 1, 1)
X_test_lstm = X_test.values.reshape(-1, 1, 1)

# Criação e treinamento do modelo LSTM
model_lstm = Sequential([
    LSTM(64, input_shape=(1, 1)),
    Dense(1, activation='sigmoid')
])

model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')  # Usando MSE para regressão
model_lstm.fit(X_train_lstm, y_train, epochs=50, batch_size=32, verbose=0)

# Realizando as previsões para o conjunto de teste
y_pred_lstm = model_lstm.predict(X_test_lstm)

# Calculando as métricas de regressão
mae_lstm = mean_absolute_error(y_test, y_pred_lstm)
mse_lstm = mean_squared_error(y_test, y_pred_lstm)

# Visualizando os resultados do modelo LSTM
print(f"Erro Médio Absoluto do modelo LSTM: {mae_lstm:.2f}")
print(f"Erro Médio Quadrático do modelo LSTM: {mse_lstm:.2f}")

4/4 [==============================] - 1s 4ms/step
Erro Médio Absoluto do modelo LSTM: 1.01
Erro Médio Quadrático do modelo LSTM: 1.01


In [16]:
#!pip install gym

In [ ]:
import numpy as np
import pandas as pd
import gym

class TradingEnv(gym.Env):
    def __init__(self, data):
        super(TradingEnv, self).__init__()
        self.data = data
        self.action_space = gym.spaces.Discrete(3)  # 0: não fazer nada, 1: comprar, 2: vender
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(1,), dtype=np.float32)
        self.reset()

    def reset(self):
        self.current_step = 0
        self.profit = 0
        self.holdings = 0
        self.initial_balance = 1000  # Defina o saldo inicial desejado
        self.balance = self.initial_balance
        return self._next_observation()

    def _next_observation(self):
        return np.array([self.data['Close'].iloc[self.current_step] / self.data['Close'].max()])

    def step(self, action):
        assert self.action_space.contains(action)

        self.current_step += 1

        if action == 1:  # Comprar
            if self.holdings == 0:
                self.holdings = self.balance / self.data['Close'].iloc[self.current_step]
                self.balance = 0

        elif action == 2:  # Vender
            if self.holdings > 0:
                self.balance = self.holdings * self.data['Close'].iloc[self.current_step]
                self.holdings = 0

        # Calcular a recompensa
        current_balance = self.balance + self.holdings * self.data['Close'].iloc[self.current_step]
        self.profit = (current_balance - self.initial_balance) / self.initial_balance

        # Verificar se o episódio terminou (pode adicionar outras condições de término)
        done = self.current_step == len(self.data) - 1

        # Definir a recompensa
        if done:
            reward = self.profit
        else:
            reward = self.profit - 0.001  # Penalidade diária para incentivar ações mais curtas

        return self._next_observation(), reward, done, {}

# Carregar os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Criação do ambiente de trading
env = TradingEnv(data)


# Implementação do algoritmo Q-Learning (com a correção do índice do estado)
def q_learning(env, num_episodes=100):
    q_table = np.zeros((len(env.observation_space.high), env.action_space.n))

    for episode in range(num_episodes):
        state = env.reset()
        done = False

        while not done:
            # Converter o estado (observation) para um índice inteiro
            state_idx = int(state[0] * len(q_table))  # Usar state[0] para obter o valor único do estado

            action = np.argmax(q_table[state_idx])
            new_state, reward, done, _ = env.step(action)

            # Converter o novo estado (new_state) para um índice inteiro
            new_state_idx = int(new_state[0] * len(q_table))  # Usar new_state[0] para obter o valor único do novo estado

            # Atualizar a tabela Q
            q_table[state_idx, action] = (1 - 0.1) * q_table[state_idx, action] + 0.1 * (reward + 0.9 * np.max(q_table[new_state_idx]))

            state = new_state

q_learning(env)

# Teste da estratégia treinada
state = env.reset()
done = False

while not done:
    action = np.argmax(q_table[state])
    state, _, done, _ = env.step(action)

final_profit = env.profit
print(f"Lucro final da estratégia: {final_profit:.2%}")

In [18]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Carregar os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Pré-processamento dos dados
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data[['Close']])
X, y = [], []
for i in range(len(data_scaled) - 1):
    X.append(data_scaled[i])
    y.append(data_scaled[i + 1][0])

X = np.array(X)
y = np.array(y)

# Divisão dos dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Reshape dos dados para o formato [amostras, tempo, características]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Criação e treinamento do modelo LSTM
model_lstm = Sequential([
    LSTM(64, input_shape=(1, 1)),
    Dense(1)
])

model_lstm.compile(optimizer='adam', loss='mean_squared_error')
model_lstm.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Realizando as previsões para o conjunto de teste
y_pred_lstm = model_lstm.predict(X_test)

# Transformando as previsões para a escala original
y_pred_lstm = scaler.inverse_transform(y_pred_lstm)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculando as métricas de regressão
mae_lstm = mean_absolute_error(y_test, y_pred_lstm)
mse_lstm = mean_squared_error(y_test, y_pred_lstm)

# Visualizando os resultados do modelo LSTM
print(f"Erro Médio Absoluto do modelo LSTM: {mae_lstm:.2f}")
print(f"Erro Médio Quadrático do modelo LSTM: {mse_lstm:.2f}")

4/4 [==============================] - 0s 2ms/step
Erro Médio Absoluto do modelo LSTM: 2.59
Erro Médio Quadrático do modelo LSTM: 11.67


In [19]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Carregar os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Pré-processamento dos dados
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data[['Close']])
X, y = [], []
look_back = 10  # Defina o tamanho da janela temporal
for i in range(len(data_scaled) - look_back):
    X.append(data_scaled[i:i + look_back])
    y.append(data_scaled[i + look_back])

X = np.array(X)
y = np.array(y)

# Divisão dos dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Criação e treinamento do modelo CNN
model_cnn = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(look_back, 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(1)
])

model_cnn.compile(optimizer='adam', loss='mean_squared_error')
model_cnn.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Realizando as previsões para o conjunto de teste
y_pred_cnn = model_cnn.predict(X_test)

# Transformando as previsões para a escala original
y_pred_cnn = scaler.inverse_transform(y_pred_cnn)
y_test = scaler.inverse_transform(y_test)

# Calculando as métricas de regressão
mae_cnn = mean_absolute_error(y_test, y_pred_cnn)
mse_cnn = mean_squared_error(y_test, y_pred_cnn)

# Visualizando os resultados do modelo CNN
print(f"Erro Médio Absoluto do modelo CNN: {mae_cnn:.2f}")
print(f"Erro Médio Quadrático do modelo CNN: {mse_cnn:.2f}")

4/4 [==============================] - 0s 3ms/step
Erro Médio Absoluto do modelo CNN: 3.12
Erro Médio Quadrático do modelo CNN: 15.16


In [20]:
#!pip install xgboost

In [21]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Carregar os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Preparação dos dados
data['Target'] = np.where(data['Close'].shift(-1) > data['Close'], 1, 0)  # Substituir os valores -1 por 0

X = data[['Close']]
y = data['Target']

# Divisão dos dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Criação e treinamento do modelo XGBoost
model_xgb = XGBClassifier()
model_xgb.fit(X_train, y_train)

# Realizando as previsões para o conjunto de teste
y_pred_xgb = model_xgb.predict(X_test)

# Avaliando o desempenho do modelo XGBoost
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
classification_report_output_xgb = classification_report(y_test, y_pred_xgb)

# Visualizando os resultados do modelo XGBoost
print(f"Acurácia do modelo XGBoost: {accuracy_xgb:.2f}")
print("Relatório de Classificação do modelo XGBoost:\n", classification_report_output_xgb)

Acurácia do modelo XGBoost: 0.47
Relatório de Classificação do modelo XGBoost:
               precision    recall  f1-score   support

           0       0.54      0.33      0.41        57
           1       0.42      0.64      0.51        44

    accuracy                           0.47       101
   macro avg       0.48      0.48      0.46       101
weighted avg       0.49      0.47      0.45       101



In [22]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Carregar os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Preparação dos dados
data['Target'] = np.where(data['Close'].shift(-1) > data['Close'], 1, -1)

X = data[['Close']]
y = data['Target']

# Divisão dos dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Criação e treinamento do modelo Random Forest
model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)

# Realizando as previsões para o conjunto de teste
y_pred_rf = model_rf.predict(X_test)

# Avaliando o desempenho do modelo Random Forest
accuracy_rf = accuracy_score(y_test, y_pred_rf)
classification_report_output_rf = classification_report(y_test, y_pred_rf)

# Visualizando os resultados do modelo Random Forest
print(f"Acurácia do modelo Random Forest: {accuracy_rf:.2f}")
print("Relatório de Classificação do modelo Random Forest:\n", classification_report_output_rf)


Acurácia do modelo Random Forest: 0.45
Relatório de Classificação do modelo Random Forest:
               precision    recall  f1-score   support

          -1       0.51      0.37      0.43        57
           1       0.40      0.55      0.46        44

    accuracy                           0.45       101
   macro avg       0.46      0.46      0.45       101
weighted avg       0.46      0.45      0.44       101



In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Carregar os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Preparação dos dados
data['Target'] = np.where(data['Close'].shift(-1) > data['Close'], 1, -1)

X = data[['Close']]
y = data['Target']

# Normalização dos dados
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

# Divisão dos dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Ajustar o tamanho do conjunto de teste para permitir a criação de sequências de tamanho 10
look_back = 10
num_samples_test = len(X_test)
num_sequences = num_samples_test - look_back + 1
X_test_new = np.zeros((num_sequences, look_back, 1))
y_test_new = np.zeros(num_sequences)

for i in range(num_sequences):
    X_test_new[i] = X_test[i:i+look_back].reshape(look_back, 1)
    y_test_new[i] = y_test[i+look_back-1]

X_test = X_test_new
y_test = y_test_new

# Criação e treinamento do modelo híbrido LSTM + CNN
model_hybrid = Sequential([
    LSTM(64, input_shape=(look_back, 1), return_sequences=True),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_hybrid.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_hybrid.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Realizando as previsões para o conjunto de teste
y_pred_hybrid = model_hybrid.predict_classes(X_test)

# Avaliando o desempenho do modelo híbrido LSTM + CNN
accuracy_hybrid = accuracy_score(y_test, y_pred_hybrid)
classification_report_output_hybrid = classification_report(y_test, y_pred_hybrid)

# Visualizando os resultados do modelo híbrido LSTM + CNN
print(f"Acurácia do modelo híbrido LSTM + CNN: {accuracy_hybrid:.2f}")
print("Relatório de Classificação do modelo híbrido LSTM + CNN:\n", classification_report_output_hybrid)

In [ ]:
import numpy as np
import pandas as pd
import gym
from gym import spaces
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque
import random

class TradingEnv(gym.Env):
    def __init__(self, data):
        super(TradingEnv, self).__init__()
        self.data = data
        self.action_space = spaces.Discrete(3)  # 0: não fazer nada, 1: comprar, 2: vender
        self.observation_space = spaces.Box(low=0, high=1, shape=(1,), dtype=np.float32)
        self.reset()

    def reset(self):
        self.current_step = 0
        self.profit = 0
        self.holdings = 0
        self.initial_balance = 1000  # Defina o saldo inicial desejado
        self.balance = self.initial_balance
        return self._next_observation()

    def _next_observation(self):
        return np.array([self.data['Close'].iloc[self.current_step] / self.data['Close'].max()])

    def step(self, action):
        assert self.action_space.contains(action)

        self.current_step += 1

        if action == 1:  # Comprar
            if self.holdings == 0:
                self.holdings = self.balance / self.data['Close'].iloc[self.current_step]
                self.balance = 0

        elif action == 2:  # Vender
            if self.holdings > 0:
                self.balance = self.holdings * self.data['Close'].iloc[self.current_step]
                self.holdings = 0

        # Calcular a recompensa
        current_balance = self.balance + self.holdings * self.data['Close'].iloc[self.current_step]
        self.profit = (current_balance - self.initial_balance) / self.initial_balance

        # Verificar se o episódio terminou (pode adicionar outras condições de término)
        done = self.current_step == len(self.data) - 1

        # Definir a recompensa
        if done:
            reward = self.profit
        else:
            reward = self.profit - 0.001  # Penalidade diária para incentivar ações mais curtas

        return self._next_observation(), reward, done, {}

# Carregar os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Criação do ambiente de trading
env = TradingEnv(data)

# Implementação do algoritmo Deep Q-Network (DQN)
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # Fator de desconto
        self.epsilon = 1.0  # Probabilidade de exploração inicial
        self.epsilon_decay = 0.995  # Taxa de decaimento da probabilidade de exploração
        self.epsilon_min = 0.01  # Probabilidade mínima de exploração
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Configuração do agente DQN
state_size = 1
action_size = 3
batch_size = 32
agent = DQNAgent(state_size, action_size)

# Treinamento do agente DQN
for e in range(100):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(500):
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("Época: {}/{}, Lucro: {:.2f}"
                  .format(e, 100, env.profit))
            break
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)

# Teste da estratégia aprendida
state = env.reset()
state = np.reshape(state, [1, state_size])
done = False
while not done:
    action = agent.act(state)
    next_state, _, done, _ = env.step(action)
    state = np.reshape(next_state, [1, state_size])

final_profit = env.profit
print(f"Lucro final da estratégia: {final_profit:.2f}")


In [ ]:
...